# 1. Diabetes 

## (1) 라이브러리 import



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

## (2) 데이터 확인 

In [ ]:
diabetes_data = load_diabetes()

In [ ]:
dir(diabetes_data)

In [ ]:
df_X = diabetes_data['data']
df_y = diabetes_data['target']
df_X.shape, df_y.shape

In [ ]:
df = pd.DataFrame(df_X, columns=diabetes_data['feature_names'])
sns.pairplot(df)

## (3) 데이터 준비하기


In [ ]:
df_X = np.array(df_X)
df_y = np.array(df_y)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size = 0.2, random_state=94)

In [ ]:
print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

## (4) 모델 준비

- W, b 초기화 
- 모델 정의 
- loss 정의 = > MSE 함수로 정의 
- Gradient 함수 구현 

In [ ]:
_, l = train_X.shape
W = np.random.rand(l)
b = np.random.rand()
W, b

In [ ]:
def model(X, W, b) :
    predictions = 0
#     for i in range(l):
#         predictions += X[:, i] * W[i]
#     predictions += b
    predictions = np.sum([X[:, i] * W[i] for i in range(l)], axis=0) + b
    
    return predictions

In [ ]:
def MSE(a, b):
    mse = ((a - b) ** 2).mean()  # 두 값의 차이의 제곱의 평균
    return mse

In [ ]:
def loss(X, W, b, y):
    predictions = model(X, W, b)
    L = MSE(predictions, y)
    return L

In [ ]:
def gradient(X, W, b, y) :
    N = len(X)
    
    y_pred = model(X, W, b)
    
    dW = 1/N * 2 * X.T.dot(y_pred-y)
    db = 2 * (y_pred-y).mean()
    
    return dW, db

In [ ]:
dW, db = gradient(df_X, W, b, df_y)
print("dW:", dW)
print("db:", db)

## (5) 학습
- Learning Rate는 0.1으로 설정.
- Iteration횟수는 충분히 많이 준다.
- MSE의 값이 3000 이하의 점수가 나올 때, 10번정도 checks 리스트에 저장을 한 후, 반복문을 종료한다.
- 이후 checks 리스트를 iteration 횟수와 함께 MSE 값을 출력해준다.

In [ ]:
LEARNING_RATE = 0.1

In [ ]:
losses = []
checks = []

for i in range(1, 100001):
    dW, db = gradient(train_X, W, b, train_y)
    W -= LEARNING_RATE * dW
    b -= LEARNING_RATE * db
    L = loss(df_X, W, b, df_y)
    losses.append(L)
    if i%10 == 0 :
        print('Iteration %d : Loss %0.4f' % (i, L))
    if L <= 3000 :
        checks.append((i, L))
        if len(checks) == 10:
            break

In [ ]:
print("MSE 3000이하\n"+'-'*30)
for num, score in checks :
    print("Iteration {} : Loss : {:.4f}".format(num, score))

## (6) 결과 예측 및 그래프 그리기 


In [ ]:
 plt.plot(losses)
plt.show()

In [ ]:
prediction = model(test_X, W, b)
mse = loss(test_X, W, b, test_y)
mse

In [ ]:
test_X.shape, test_y.shape
type(test_y)

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(15, 10))
plt.scatter(test_X[:, 0], test_y, label="label")
plt.scatter(test_X[:, 0], prediction, label="pred")
plt.legend()

plt.show()

# 월요일 오후 3시, 자전거 타기 

## (1) import


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

## (2) 데이터 준비 
- train.csv
- test.csv
- sampleSubmission.csv 
불러오기 


In [ ]:
import pandas as pd 
from google.colab import files 
files.upload()

In [ ]:
train = pd.read_csv('train.csv')

train.head()


In [ ]:
test = pd.read_csv('test.csv')

test

- 데이터 분포 시각화 

In [ ]:
sns.pairplot(train)


## datetime 컬럼을 datetime 자료형으로 변환

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'])
train.head()


In [ ]:
date_col = ['year', 'month', 'day', 'hour', 'minute', 'second']

for col_name in date_col :
    
    if col_name == 'year':
        train[col_name] = train['datetime'].dt.year
    elif col_name == 'month':
        train[col_name] = train['datetime'].dt.month
    elif col_name == 'day':
        train[col_name] = train['datetime'].dt.day
    elif col_name == 'hour':
        train[col_name] = train['datetime'].dt.hour
    elif col_name == 'minute':
        train[col_name] = train['datetime'].dt.minute
    elif col_name == 'second':
        train[col_name] = train['datetime'].dt.second
        
# map(lambda) 사용하기 => dt빼고

train.head()

## 데이터 개수 시각화 

- sms.countplt, subplot 활용
- month,. day, hour 는 막대 그래프 상위에 값 출력


In [ ]:
plt.figure(figsize=(30, 20))
for i in range(len(date_col)) :
    plt.subplot(3, 2, i+1)
    ax = sns.countplot(train[date_col[i]])
    
    if i not in  [0, 4, 5]:
        for p in ax.patches:
            ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+10))

## 데이터 분리 
- info() 를 통해 문자열 데이터인지를 확인
- train ['datetime'].dt.weekday 활용, 요일에 대한 정보 추가 
- 학습 데이터 및 RMSE 값:

- 요일, 시간, 날씨 : 167.031365422865

- 요일, 시간, 날씨 + temp : 155.26878251118987

- 요일, 시간, 날씨 + humidity : 163.28265800809078

- 요일, 시간, 날씨 + humidity + temp,atemp + season : 149.625184945586


In [ ]:
train.info()

In [ ]:
# 요일 정보
train['datetime'].dt.weekday

# (3) 학습 및 예측

## model_train 함수 작성 
- train_test_split 을 사용하여 데이터셋 나누기 
- LinearRegression 모델 생성 
- 학습 
- 예측 및 확인 

In [ ]:
def model_train(X, y) :
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=94)
    
    model = LinearRegression()
    model.fit(X_train, y_train)

    
    y_pred = model.predict(X_test)
    score = mean_squared_error(y_test, y_pred)
    
    return score, np.sqrt(score)



In [ ]:
# 요일 시간 날씨
day = train['datetime'].dt.weekday
X = pd.concat([train['hour'], train['weather'], train['datetime'].dt.weekday], axis=1, ignore_index=False)
y = train['count']

In [ ]:
X

In [ ]:
mse_score, rmse_score = model_train(X, y)
print("MSE : {:.4f}\nRMSE : {:.4f}".format(mse_score, rmse_score))

In [ ]:
# 요일 시간 날씨 + temp
X = pd.concat([train['hour'], train['weather'], train['datetime'].dt.weekday, train['temp']], axis=1, ignore_index=False)
X

In [ ]:
mse_score, rmse_score = model_train(X, y)
print("MSE : {:.4f}\nRMSE : {:.4f}".format(mse_score, rmse_score))

In [ ]:
# 요일 시간 날씨 + humidity
X = pd.concat([train['hour'], train['weather'], train['datetime'].dt.weekday, train['humidity']], axis=1, ignore_index=False)
X 

In [ ]:
mse_score, rmse_score = model_train(X, y)
print("MSE : {:.4f}\nRMSE : {:.4f}".format(mse_score, rmse_score))

In [ ]:
# 요일 시간 날씨 + humidity + temp
X = pd.concat([train['hour'], train['weather'], train['datetime'].dt.weekday, train['humidity'], train['temp']], axis=1, ignore_index=False)
X

In [ ]:
mse_score, rmse_score = model_train(X, y)
print("MSE : {:.4f}\nRMSE : {:.4f}".format(mse_score, rmse_score))

In [ ]:
# 요일 시간 날씨 + humidity + temp, atemp + season
X = pd.concat([train['hour'], train['weather'], train['datetime'].dt.weekday, train['humidity'], train['temp'], train['atemp'], train['season']], axis=1, ignore_index=False)
X

In [ ]:
mse_score, rmse_score = model_train(X, y)
print("MSE : {:.4f}\nRMSE : {:.4f}".format(mse_score, rmse_score))  

# 예측결과 시각화 

- 결과 중 가장 낮은 예측 결과로 사용 

- x축 : X 데이터 중 temp데이터 / y축 : count
- x축 : X 데이터 중 humidity데이터 / y축 : count

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(15, 10))
plt.scatter(y_test, X_test['temp'], label="label")
plt.scatter(y_pred, X_test['temp'], label="pred")
plt.legend()

plt.show()

In [ ]:
plt.style.use('ggplot')
plt.figure(figsize=(15, 10))
plt.scatter(y_test, X_test['humidity'], label="label")
plt.scatter(y_pred, X_test['humidity'], label="pred")
plt.legend()

plt.show()

# 회고 


## 당뇨병 예측 

- Learning Rate 은 0.1 로 설정 
- loss : MSE
- iteration 2851회 부터 MSE 손실함수값은 3000 이하로 떨어짐을 확인 : 예측데이터 시각화 
- test 데이터에 대한 성능 : 2870.4427694147507





## 자전거 

### - 학습 데이터 별 RMSE값

- 요일, hour, weather : RMSE = 155.2688

- 요일, hour, weather, tmp : RMSE = 155.2688

- 요일, hour, weather, humidity : RMSE = 163.2827

- 요일, hour, weather, humidity, temp : RMSE = 151.4418

- 요일, hour, weather, humidity, temp, atemp, season : RMSE =  149.6252


- 5번째 학습데이터 RMSE 값 150 이하를 달성 : 예측 결과 데이터 시각화

-     1. x축 : X 데이터 중 temp데이터 / y축 : count

-     2. x축 : X 데이터 중 humidity데이터 / y축 : count


